In [1]:
import pandas as pd

# Load the training and test data
train_df = pd.read_csv('C:/Users/krish/Downloads/HV-AI-2024/HV-AI-2024/train.csv')
test_df = pd.read_csv('C:/Users/krish/Downloads/HV-AI-2024/HV-AI-2024/test.csv')


In [2]:
print(train_df.head())
print(test_df.head())


                   path  class                    bbox
0  images/train/1_2.jpg      0  139.0 30.0 153.0 264.0
1  images/train/1_4.jpg      0  112.0 90.0 255.0 242.0
2  images/train/1_5.jpg      0   70.0 50.0 134.0 303.0
3  images/train/1_7.jpg      0    7.0 75.0 420.0 262.0
4  images/train/1_8.jpg      0   78.0 86.0 333.0 158.0
                        path                     bbox
0    images/test/test_81.jpg  241.0 113.0 202.0 257.0
1  images/test/test_1592.jpg  119.0 121.0 294.0 337.0
2   images/test/test_565.jpg    33.0 45.0 237.0 139.0
3  images/test/test_2078.jpg    58.0 95.0 262.0 210.0
4  images/test/test_3645.jpg   32.0 141.0 348.0 281.0


In [3]:
import cv2
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the base directory for images
base_dir = 'C:/Users/krish/Downloads/HV-AI-2024/HV-AI-2024/'

# Update paths to include the full directory
train_df['path'] = base_dir + train_df['path']
test_df['path'] = base_dir + test_df['path']

# Function to load and preprocess images
def load_and_preprocess_image(image_path, target_size=(224, 224)):
    img = cv2.imread(image_path)
    if img is None:
        return None
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, target_size)
    img = img / 255.0  # Normalize to [0, 1]
    return img

# Apply preprocessing to the train and test data
train_df['processed_images'] = train_df['path'].apply(load_and_preprocess_image)
test_df['processed_images'] = test_df['path'].apply(load_and_preprocess_image)

# Drop rows where image loading failed (if any)
train_df = train_df.dropna(subset=['processed_images'])
test_df = test_df.dropna(subset=['processed_images'])


In [4]:
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Convert class labels to strings for the generator
train_df['class'] = train_df['class'].astype(str)

# Create the data generators
train_generator = datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='path',
    y_col='class',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=True
)


Found 5994 validated image filenames belonging to 200 classes.


In [5]:
IMG_WIDTH =3
IMG_HEIGHT = 3

In [6]:
from tensorflow.keras import layers, models, optimizers

# Define the CNN model architecture
def build_cnn_model(input_shape, num_classes):
    model = models.Sequential()
    
    # First Conv Block
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.Conv2D(32, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(0.25))
    
    # Second Conv Block
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(0.25))
    
    # Third Conv Block
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(0.25))
    
    # Flatten and Dense Layers
    model.add(layers.Flatten())
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(num_classes, activation='softmax'))
    
    return model

# Define the input shape (e.g., 224x224x3 for colored images)
input_shape = (224, 224, 3)  # Adjust this according to your image size

# Number of classes
num_classes = len(train_df['class'].unique())

# Build the model
cnn_model = build_cnn_model(input_shape, num_classes)

# Compile the model
cnn_model.compile(optimizer=optimizers.Adam(learning_rate=1e-4), 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])

# Print the model summary
cnn_model.summary()


c:\Users\krish\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 220, 220, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 110, 110, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 110, 110, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 108, 108, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 106, 106, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 53, 53, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 53, 53, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 51, 51, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 49, 49, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 24, 24, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 24, 24, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 73728)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │    37,749,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 200)            │       102,600 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 38,138,856 (145.49 MB)

 Trainable params: 38,138,856 (145.49 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:
# Train the model
cnn_model.fit(
    train_generator,
    epochs=1,  # You can increase the number of epochs for better results
    steps_per_epoch=len(train_generator)
)


142/188 ━━━━━━━━━━━━━━━━━━━━ 2:12 3s/step - accuracy: 0.0032 - loss: 5.2983

In [5]:
# from tensorflow.keras.applications import EfficientNetB0
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
# from tensorflow.keras.optimizers import Adam

# # Load the EfficientNetB0 model without the top layer
# base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# # Add custom layers on top of the base model
# x = base_model.output
# x = GlobalAveragePooling2D()(x)
# x = Dense(1024, activation='relu')(x)
# predictions = Dense(len(train_df['class'].unique()), activation='softmax')(x)  # Adjust the output layer for the number of classes

# # Define the final model
# model = Model(inputs=base_model.input, outputs=predictions)

# # Compile the model
# model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

# # Print model summary
# model.summary()


ImportError: cannot import name 'EfficientNetB0'

In [ ]:
import numpy as np

# Stack the processed test images into a NumPy array
test_images = np.stack(test_df['processed_images'].values)

# Make predictions
predictions = cnn_model.predict(test_images)

# Extract the predicted classes and confidence scores
test_df['predicted_label'] = np.argmax(predictions, axis=1)
test_df['confidence_score'] = np.max(predictions, axis=1)


In [ ]:
# Prepare the submission dataframe
submission_df = test_df[['path', 'predicted_label', 'confidence_score']]

# Save the predictions to a CSV file
submission_df.to_csv('predictions.csv', index=False)
